In [1]:
import wandb

In [2]:
run = wandb.Api().run("/silvan-wiedmer-fhgr/VisionTransformer/runs/3s7lhtef")

In [3]:
run.summary

{'_runtime': 552.536836667, '_step': 101, '_timestamp': 1727429122.0140896, '_wandb': {'runtime': 552}, 'confusion-matrix': {'_type': 'image-file', 'format': 'png', 'height': 480, 'path': 'media/images/confusion-matrix_101_bbae2909cf13e9de65d4.png', 'sha256': 'bbae2909cf13e9de65d44712ed2323716c5b17cc007d136c6e2e572c2594ff81', 'size': 24283, 'width': 640}, 'epoch/accuracy': 1, 'epoch/epoch': 99, 'epoch/learning_rate': 0.0010000000474974513, 'epoch/loss': 0.3520642817020416, 'epoch/val_accuracy': 0.8888888955116272, 'epoch/val_loss': 0.3337496817111969, 'test_accc': 0.8253424763679504, 'test_loss': 0.453955203294754}

In [4]:
history = run.history()
history

,epoch/val_accuracy,epoch/learning_rate,epoch/accuracy,epoch/val_loss,_step,_runtime,test_loss,epoch/loss,_timestamp,confusion-matrix,epoch/epoch,test_accc
0,0.526786,0.001,0.486096,0.689646,0,18.109223,NaN,1.002055,1.727429e+09,None,0.0,NaN
1,0.666667,0.001,0.562500,0.677286,1,18.280962,NaN,0.688837,1.727429e+09,None,1.0,NaN
2,0.544643,0.001,0.506118,0.691357,2,29.311693,NaN,0.692821,1.727429e+09,None,2.0,NaN
3,0.222222,0.001,0.468750,0.693214,3,29.466898,NaN,0.691901,1.727429e+09,None,3.0,NaN
4,0.540179,0.001,0.537264,0.685560,4,40.290938,NaN,0.692656,1.727429e+09,None,4.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
97,0.777778,0.001,0.718750,0.430567,97,539.647954,NaN,0.504339,1.727429e+09,None,97.0,NaN
98,0.785714,0.001,0.757543,0.470704,98,550.599944,NaN,0.495521,1.727429e+09,None,98.0,NaN
99,0.888889,0.001,1.000000,0.333750,99,550.668174,NaN,0.352064,1.727429e+09,None,99.0,NaN
100,NaN,NaN,NaN,NaN,100,551.152930,0.453955,NaN,1.727429e+09,None,NaN,0.825342


In [5]:
max(history["test_accc"])

nan

In [6]:
import plotly.express as px

In [7]:
px.line(history["epoch/val_accuracy"])

# Sweep

In [8]:
api = wandb.Api()

In [9]:
cnn_sweep = api.sweep("silvan-wiedmer-fhgr/VisionTransformer/j89j3ya1")

In [10]:
cnn_sweep.runs

<Runs silvan-wiedmer-fhgr/VisionTransformer>

In [26]:
METRIC = "epoch/val_accuracy"

In [27]:
runs = sorted(cnn_sweep.runs, key=lambda run: run.summary.get(METRIC, 0), reverse=True)
val_acc = runs[0].summary.get(METRIC, 0)
print(f"Best run {runs[0].name} with {val_acc} validation accuracy")


Best run lr_0.0005758520600362071_bs_16_do_0.36 with 0.53125 validation accuracy


In [28]:
runs[0].config

{'dropout': 0.3628469208876618,
 'batch_size': 16,
 'learning_rate': 0.0005758520600362071}

In [31]:
df_system_metrics = runs[0].history(stream="system")
df_system_metrics.head()

,system.network.sent,system.network.recv,system.disk./.usageGB,_wandb,system.disk.out,_runtime,system.disk./.usagePercent,system.disk.in,_timestamp,system.gpu.0.powerPercent,...,system.cpu,system.proc.cpu.threads,system.cpu.0.cpu_percent,system.cpu.19.cpu_percent,system.cpu.13.cpu_percent,system.cpu.10.cpu_percent,system.cpu.8.cpu_percent,system.cpu.18.cpu_percent,system.cpu.17.cpu_percent,system.cpu.14.cpu_percent
0,80128.0,590910.0,455.232357,True,0.0,10.293340,48.924321,0.0,1.728653e+09,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,True,NaN,10.306290,NaN,NaN,1.728653e+09,54.449091,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,True,NaN,10.378034,NaN,NaN,1.728653e+09,NaN,...,21.81267,120.0,1.426534,8.238636,8.392603,11.697575,9.82906,8.238636,8.238636,8.522727
3,NaN,NaN,455.231991,True,0.0,20.259459,48.924282,0.0,1.728653e+09,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,True,NaN,20.260639,NaN,NaN,1.728653e+09,54.449091,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
df_system_metrics.columns

Index(['system.network.sent', 'system.network.recv', 'system.disk./.usageGB',
       '_wandb', 'system.disk.out', '_runtime', 'system.disk./.usagePercent',
       'system.disk.in', '_timestamp', 'system.gpu.0.powerPercent',
       'system.gpu.0.smClock', 'system.gpu.0.powerWatts', 'system.gpu.0.temp',
       'system.gpu.0.memory', 'system.memory_percent', 'system.gpu.0.gpu',
       'system.proc.memory.rssMB', 'system.gpu.0.memoryAllocatedBytes',
       'system.proc.memory.availableMB', 'system.proc.memory.percent',
       'system.gpu.0.memoryAllocated', 'system.gpu.0.memoryClock',
       'system.gpu.0.enforcedPowerLimitWatts', 'system.cpu.9.cpu_percent',
       'system.cpu.1.cpu_percent', 'system.cpu.7.cpu_percent',
       'system.cpu.16.cpu_percent', 'system.cpu.6.cpu_percent',
       'system.cpu.11.cpu_percent', 'system.cpu.3.cpu_percent',
       'system.cpu.15.cpu_percent', 'system.cpu.4.cpu_percent',
       'system.cpu.12.cpu_percent', 'system.cpu.2.cpu_percent',
       'system.cpu

In [33]:
df_system_metrics["system.cpu"]

0           NaN
1           NaN
2     21.812670
3           NaN
4           NaN
        ...    
68          NaN
69    21.596272
70          NaN
71    21.550835
72          NaN
Name: system.cpu, Length: 73, dtype: float64